# Labs: Efficient GMean and GMeanChanges

## 1. Imports

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import gmean
from quantopian.pipeline.data.morningstar import Fundamentals
from quantopian.pipeline.filters import StaticAssets, QTradableStocksUS
from quantopian.pipeline.factors import CustomFactor
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

# MEOH has no update starting from 2017-12-31, so it's great to include here.
# universe = StaticAssets(symbols(['FB', 'MEOH', 'MCD', 'MAC']))

# Single stock
# universe = StaticAssets(symbols(['FB']))

# The real test
universe = QTradableStocksUS()

mask = {'mask': universe}

start_date = '2020-03-23'
end_date = '2020-03-24'

## 2. Functions

In [8]:
# Todo: Create a working get_periodical() function

def get_periodical(asof_date, values, period_skip=4, num_periods=2):
    """ Get unique values
    Set period_skip to 1 for quarterly, 4 for yearly/annual.
    """
    return values # No need to return the dates

True

## 3. CustomFactors

In [9]:
def get_changes(data, t=1):
    data = pd.Series(data)
    prev_data = data.shift(t)
    return data/prev_data

# Todo: Create GMean and GMeanChanges CustomFactors


In [10]:
# Todo: Must return the same result with the Loop-by-Asset version

def make_pipeline():
    period_skip = 4
    num_periods = 8
    gm = GMean(
        inputs=[Fundamentals.gross_margin_asof_date,
                Fundamentals.gross_margin],
        window_length=int((252/4*period_skip+1)*num_periods),
        period_skip=period_skip,
        num_periods=num_periods,
        mask=universe
    )
    gm_growth = GMeanChanges(
        inputs=[Fundamentals.gross_margin_asof_date,
                Fundamentals.gross_margin],
        window_length=int((252/4*period_skip+1)*num_periods),
        period_skip=period_skip,
        num_periods=num_periods,
        mask=universe
    ) 
    
    return Pipeline(columns={'gm_8yr': gm,
                             'gm_growth_8yr': gm_growth,
                             'asof_date': Fundamentals.gross_margin_asof_date.latest},
                    screen = universe)


pipe = make_pipeline()
mdf = run_pipeline(pipe, start_date, end_date).dropna(how='all')
mdf.head(10)

/venvs/py35/lib/python3.5/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: In the future, NAT != NAT will be True rather than False.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
/venvs/py35/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


asof_date    gm_8yr  \
2020-03-23 00:00:00+00:00 Equity(2 [ARNC])   2019-12-31  0.192367   
                          Equity(24 [AAPL])  2019-12-31  0.387228   
                          Equity(41 [ARCB])  2019-12-31       NaN   
                          Equity(52 [ABM])   2020-01-31  0.100725   
                          Equity(53 [ABMD])  2019-12-31  0.824567   
                          Equity(62 [ABT])   2019-12-31  0.554794   
                          Equity(64 [GOLD])  2019-09-30  0.333868   
                          Equity(67 [ADSK])  2020-01-31  0.873852   
                          Equity(76 [TAP])   2019-12-31  0.374520   
                          Equity(110 [RAMP]) 2019-12-31  0.439466   

                                              gm_growth_8yr  
2020-03-23 00:00:00+00:00 Equity(2 [ARNC])         1.048445  
                          Equity(24 [AAPL])        0.999133  
                          Equity(41 [ARCB])             NaN  
                          Equity(52 [ABM])         1.020498  
                          Equity(53 [ABMD])        1.005854  
                          Equity(62 [ABT])         1.031920  
                          Equity(64 [GOLD])        0.964688  
                          Equity(67 [ADSK])        1.001721  
                          Equity(76 [TAP])         1.009523  
                          Equity(110 [RAMP])       1.156292